In [1]:
link1 = 'faculdade-projecao/lista-reclamacoes/'
link2 = 'faculdade-projecao/lista-reclamacoes/?pagina=2'
link3 = 'faculdade-projecao/lista-reclamacoes/?pagina=3'

In [2]:
import time
from dotenv import load_dotenv
import os
import pyautogui as pg
import pandas as pd
from botcity.maestro import *
from botcity.core import DesktopBot
from botcity.web import WebBot, Browser, By
import requests as rq

In [3]:
pagina_scraping = pg.confirm(
    title='Inicio do Scraping', 
    text='Escolha uma opção a seguir:',
    buttons=['Página 1','Página 2', 'Página 3','Cancelar']    
)

match pagina_scraping:
    case 'Página 1':
        url_scraping = f'https://www.reclameaqui.com.br/empresa/{link1}'
    case 'Página 2':
        url_scraping = f'https://www.reclameaqui.com.br/empresa/{link2}'
    case 'Página 3':
        url_scraping = f'https://www.reclameaqui.com.br/empresa/{link3}'


In [4]:
botweb = WebBot()
botweb.headless = True
botweb.browser = Browser.CHROME
botweb.driver_path = "chromedriver.exe"

titulo_reclamacoes = []
descricao_reclamacoes = []
link_reclamacoes = []
resposta_reclamacoes = []
hora_reclamacoes = []

def extrair_informacoes(xpath, atributo, lista_destino):
    elementos = botweb.find_elements(xpath, by=By.XPATH)
    for elemento in elementos:
        lista_destino.append(elemento.get_attribute(atributo))

botweb.browse(url_scraping)
time.sleep(2)
print("Iniciando o scraping")
    
try:
    extrair_informacoes('//h4[@class="sc-1pe7b5t-1 bVKmkO"]', 'textContent', titulo_reclamacoes)
    extrair_informacoes('//p[@class="sc-1pe7b5t-2 eHoNfA"]', 'textContent', descricao_reclamacoes)
    extrair_informacoes('//div[@class="sc-1pe7b5t-0 eJgBOc"]//a', 'href', link_reclamacoes)
    extrair_informacoes('//div[@class="sc-1pe7b5t-3 gkhqVK"]//span[1]', 'textContent', resposta_reclamacoes)
    extrair_informacoes('//div[@class="sc-1pe7b5t-3 gkhqVK"]//span[2]', 'textContent', hora_reclamacoes)

except Exception as e:
    print(f"Houve um erro em {e}")
    botweb.close_page()

finally:
    print("Scraping finalizado")
    botweb.close_page()

Iniciando o scraping
Scraping finalizado


In [6]:
df = pd.DataFrame({
    'titulo_reclamacao': titulo_reclamacoes,
    'decricao': descricao_reclamacoes,
    'links': link_reclamacoes,
    'resposta_empresa': resposta_reclamacoes,
    'hora_reclamacao': hora_reclamacoes    
})

json_data = df.to_json(orient='records', indent=1)
json_data = json_data.replace("\\/","/")

In [7]:
data_to_send = {
    "dados_gerais": json_data,
    "whatsapp":"sim"
}

load_dotenv()
url = os.getenv('url_wh')

response = rq.post(url, data=data_to_send)

print("Status code: ", response.status_code)
print("Resposta do Servidor: ", response.json())

Status code:  200
Resposta do Servidor:  {'message': 'Workflow was started'}
